In [1]:
from pathlib import Path
from typing import Union
import itk
import os
import time

In [2]:
BASE_PATH = Path(os.getcwd())
BASE_PATH

PosixPath('/home/estudiante/curso-imageproc/medical-image-processing-project')

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_array(arr, cmap='viridis', title='Array Visualization'):
    """
    Visualizes a 2D numpy array using imshow.

    Parameters:
    - arr: 2D numpy array
    - cmap: Colormap for visualization (default: 'viridis')
    - title: Title of the plot
    """
    if arr.ndim != 2:
        raise ValueError("Input array must be 2D")

    plt.figure(figsize=(6, 6))
    plt.imshow(arr, cmap=cmap)
    plt.colorbar(label='Value')
    plt.title(title)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.tight_layout()
    plt.show()

In [35]:
names = ["10", "11", "12", "14", "16", "17", "18", "28"]
name = names[0]
extension = "nii.gz"
input_image = BASE_PATH / f"train_nifti/{name}/p{name}_PET.{extension}"

In [36]:
dimensions = 3
pixel_type_in = itk.F
pixel_type_out = itk.UC
image_type_in = itk.Image[pixel_type_in, dimensions]
image_type_out = itk.Image[pixel_type_out, dimensions]

# Step 1: Read the image as float
reader = itk.ImageFileReader[image_type_in].New()
reader.SetFileName(str(input_image))
reader.Update()

image = reader.GetOutput()
size = itk.size(image)
np_image = itk.GetArrayFromImage(image)
np_image.shape

(241, 144, 144)

In [6]:
ct_spacing = image.GetSpacing()
ct_origin = image.GetOrigin()
ct_direction = image.GetDirection()

In [7]:
ct_spacing

itkVectorD3 ([1.17188, 1.17188, 1.5])

In [8]:
ct_origin

itkPointD3 ([-300, -230, -1125.18])

In [18]:
ct_direction

itkMatrixD33 ([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])

In [30]:
ct_direction_array = ct_direction.__array__()

In [32]:
ct_direction_array[2][2]

np.float64(1.0)

In [10]:
from scripts.slices import extract_initial_slice, extract_final_slice

initial_slice = extract_initial_slice(np_image, intensity_threshold=50, min_sequence_lenght=10)
initial_slice

364

In [34]:
z_world = ct_origin[2] + initial_slice * ct_spacing[2] * ct_direction_array[2][2]
z_world

np.float64(-579.1800537109375)

In [38]:
pet_spacing = image.GetSpacing()
pet_origin = image.GetOrigin()
pet_direction = image.GetDirection().__array__()

In [40]:
pet_slice_index = int(round((z_world - pet_origin[2]) / (pet_spacing[2] * pet_direction[2][2])))
pet_slice_index

136

In [ ]:
from scripts.slices import get_pet_slice_index_from_ct_slice
ct_image = BASE_PATH / f"train_nifti/{name}/p{name}_CT.{extension}"
pet_image = BASE_PATH / f"train_nifti/{name}/p{name}_PET.{extension}"
get_pet_slice_index_from_ct_slice(ct_image, pet_image, initial_slice)

In [ ]:
leg_slice, other = extract_final_slice(np_image, initial_slice, intensity_threshold=3, leg_intensity_threshold=50)
print(leg_slice, other)

In [ ]:
visualize_array(np_image[initial_slice,:,:])

In [ ]:
np_image_2 = np_image[0:initial_slice+1,:,:]

In [ ]:
np_image_2.shape

In [ ]:
from scripts.slices import kmeans_labels
num_pixels_arr = []
for z in range(np_image_2.shape[0]):
    slice_array = np_image_2[z,:,:]
    num_pixels_arr.append((slice_array > 100).sum())
    
num_pixels_arr = np.array(num_pixels_arr).reshape(-1,1)
labels = kmeans_labels(num_pixels_arr, n_clusters=3)
for i,val in enumerate(labels):
    print(i, " : ", val)
    

In [ ]:
visualize_array(np_image[141,:,:])